In [1]:
# Doing by file until I can figure out OAuth
import os
token_file = os.path.join(os.path.expanduser("~/"), ".jira_auth")
with open(token_file, 'r') as fd:
    uname = fd.readline().strip()  # Can't hurt to be paranoid
    pwd = fd.readline().strip()

SERVER = "https://jira.lsstcorp.org/"
from jira import JIRA
js = JIRA(server=SERVER, basic_auth=(uname, pwd))

In [2]:
query = "project = Simulations AND issuetype = Epic AND summary ~ \"SOCS Release\" ORDER BY key"
issues = js.search_issues(query)
print len(issues)

12


In [3]:
def get_linked_epics(issue, linkTypeName="Containment"):
    linked_epics = []
    links = issue.fields.issuelinks
    for link in links:
        if link.type.name == linkTypeName:
            linked_epics.append(js.issue(link.outwardIssue.key))
    return linked_epics

In [4]:
def create_list_from_numbered_description(desrc, bulletType="*"):
    import re
    olist = []
    for line in desrc.strip().split(os.linesep):
        values = line.split()
        if re.match("\d+\.\d+", values[0]) is None:
            numbered = bulletType
        else:
            numbered = bulletType * 2
        olist.append("{0} {1}".format(numbered, " ".join(values[1:])))
    return olist

In [6]:
SHOW_ISSUES = True
# Write some boiler plate for the page
page_content = []
page_content.append("This page details the SOCS workplan.")
import time
page_content.append("Updated: {0}".format(time.strftime("%Y-%m-%d %H:%M", time.localtime())))
page_content.append("")

for issue in issues:
    socs_summary = issue.fields.summary
    page_content.append("h2. {0}".format(socs_summary))
    socs_duedate = issue.fields.duedate
    page_content.append("*Release Date:* {0}".format(socs_duedate))
    page_content.append("")
    
    page_content.append("h3. Statement of Work")
    socs_descr = issue.fields.description
    socs_work = create_list_from_numbered_description(socs_descr)
    page_content.append(os.linesep.join(socs_work))
    page_content.append("")
    
    sub_epics = get_linked_epics(issue)
    for sub_epic in sub_epics:
        page_content.append("h4. {0}".format(sub_epic.fields.summary))
        page_content.append("h5. Statement of Work")
        if sub_epic.fields.description is None:
            descr = "No description provided."
        else:
            descr = sub_epic.fields.description
        page_content.append(descr)
        page_content.append("")
        if SHOW_ISSUES:
            page_content.append("h6. Issues")
            se_issues = js.search_issues("'Epic Link' = '{0}'".format(sub_epic.fields.summary))
            page_content.append("Number of Issues = {0}".format(len(se_issues)))
            if len(se_issues) > 0:
                page_content.append("")
                page_content.append("|| Key || Description || Status ||")
                for se_issue in se_issues:
                    page_content.append("| {0} | {1} | {2} |".format(se_issue.key, se_issue.fields.summary,
                                                                    se_issue.fields.status.name))
                
print os.linesep.join(page_content)    

This page details the SOCS workplan.
Updated: 2015-08-05 12:56

h2. SOCS Release v0.1
*Release Date:* 2015-07-31

h3. Statement of Work
* Release OpSim Requirements document
* Design SOCS independant of OpSim
* Define and initiate SOCS code modules in SE Simulations repository
* Prototype and benchmark Communications using current release of DDS-SAL-Python

h4. OpSim Requirements Document
h5. Statement of Work
* Write OpSim requirements document
* Ensure Scheduler requirements document reflects necessary interface

h6. Issues
Number of Issues = 10

|| Key || Description || Status ||
| SIM-1274 | Ensure requirements are captured in both documents | Done |
| SIM-1273 | Arrange and hold meeting to review Scheduler Requirement doc. | Done |
| SIM-1272 | Coordinate creation of diagram & narrative with Francisco | Done |
| SIM-1270 | Add text from Sphinx on DB Architecture to Appendix C  | To Do |
| SIM-1269 | Add text for configuration tables to Appendix B (parameters) | In Progress |
| SIM